# Exercise 1
## Import of packages

In [49]:
import os
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil
import random


## Organising the folders

In [50]:
all_data = {}

characters = ['bart_simpson', 'charles_montgomery_burns', 'homer_simpson', 'krusty_the_clown', 
              'lisa_simpson', 'marge_simpson', 'milhouse_van_houten', 'moe_szyslak', 'ned_flanders', 'principal_skinner']
base_dir = '/home/labuser/all_data/'
for character in characters: 
      all_data[character] = {}
      base_dir_ch = base_dir
      all_data[character]['train'] =  os.path.join(base_dir_ch, 'train/' + character)
      all_data[character]['test'] =  os.path.join(base_dir_ch, 'test/' + character)
      all_data[character]['val'] =  os.path.join(base_dir_ch, 'val/' + character)

In [51]:

for ch in characters: 
      data = all_data[ch]
      print(ch)
      for k in data.keys(): 
          print(f'total {k} images:', len(os.listdir(data[k])))
      print('-'*20)


bart_simpson
total train images: 1006
total test images: 135
total val images: 201
--------------------
charles_montgomery_burns
total train images: 894
total test images: 120
total val images: 179
--------------------
homer_simpson
total train images: 1684
total test images: 225
total val images: 337
--------------------
krusty_the_clown
total train images: 904
total test images: 121
total val images: 181
--------------------
lisa_simpson
total train images: 1015
total test images: 136
total val images: 203
--------------------
marge_simpson
total train images: 968
total test images: 130
total val images: 193
--------------------
milhouse_van_houten
total train images: 809
total test images: 108
total val images: 162
--------------------
moe_szyslak
total train images: 1089
total test images: 146
total val images: 217
--------------------
ned_flanders
total train images: 1090
total test images: 146
total val images: 218
--------------------
principal_skinner
total train images: 895
to

## Building a CNN - Model 1

In [52]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [53]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.1))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.1))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.1))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.1))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

## Model Compiling

In [54]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

## Data PreProcessing

In [55]:
# Rescale all images by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [56]:
base_dir = '/home/labuser/all_data/'

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [57]:
# Resize all images to 150 X 150 (This is the parameter that we passed to our convnet)
train_generator = train_datagen.flow_from_directory(
        train_dir, # target directory
        class_mode = 'categorical',
        target_size=(150, 150),
        batch_size=20) # Since we use binary_crossentropy loss, we need binary labels

Found 10354 images belonging to 10 classes.


In [58]:
validation_generator = validation_datagen.flow_from_directory(
        validation_dir, # validation directory
        target_size=(150, 150),
        batch_size=20,
        class_mode = 'categorical')

Found 2070 images belonging to 10 classes.


In [59]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 320, 480, 3)
labels batch shape: (20, 10)


## Callbacks

In [60]:
earlystop = EarlyStopping(
    monitor='val_acc',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto')

In [61]:
csv_logger = CSVLogger('training.log', 
                       separator=',', 
                       append=False)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=[earlystop, csv_logger]
)


Epoch 1/30
100/100 [==============================] - 31s 281ms/step - loss: 2.2025 - acc: 0.2025 - val_loss: 2.0661 - val_acc: 0.2770
Epoch 2/30
100/100 [==============================] - 30s 296ms/step - loss: 1.8501 - acc: 0.3551 - val_loss: 1.7759 - val_acc: 0.4300
Epoch 3/30
100/100 [==============================] - 28s 280ms/step - loss: 1.6590 - acc: 0.4468 - val_loss: 1.7192 - val_acc: 0.4280
Epoch 4/30
100/100 [==============================] - 28s 279ms/step - loss: 1.5310 - acc: 0.5030 - val_loss: 1.5438 - val_acc: 0.5280
Epoch 5/30
100/100 [==============================] - 28s 279ms/step - loss: 1.4356 - acc: 0.5305 - val_loss: 1.4653 - val_acc: 0.5180
Epoch 6/30
100/100 [==============================] - 28s 280ms/step - loss: 1.3653 - acc: 0.5620 - val_loss: 1.3768 - val_acc: 0.5860
Epoch 7/30
100/100 [==============================] - 28s 278ms/step - loss: 1.2931 - acc: 0.5712 - val_loss: 1.3541 - val_acc: 0.5810
Epoch 8/30
100/100 [==============================] - 2

In [ ]:
model.save('simpson.h6')

In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Transfer Learning

Purpose: Has not been applied data augumentation, which could be applied due to a reduced data training sample that causes overfitting.
By the latter graphs we can infer that we do not encounter overfitting, however, through transfer learning, we will keep the already trained weights and try to apply dropout and data aumentation.

In [35]:
! pip install keras --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [38]:
keras.__version__

'2.5.0'

In [44]:
from keras.layers import RandomFlip
from keras.layers import RandomRotation
from keras.layers import Rescaling

In [ ]:
''' This has to be substituted with our model once we understand from the professor 
with what criterion the layers.trainable = False are selected '''

base_model = keras.applications.ResNet50(
    weights='imagenet',
    input_shape=(150, 150, 3),
    include_top=False,
)

# freeze the conv-net structure
base_model.trainable = False

In [ ]:
data_augmentation = keras.Sequential(
    [
       RandomFlip("horizontal"),
       RandomRotation(0.1),
    ]
)

In [ ]:
inputs = keras.Input(shape=(150, 150, 3), name="input")
# Augment your inputs here!
x = data_augmentation(inputs)

# scale pixels here
scale_layer = Rescaling(scale=1/127.5, offset=-1)
x = scale_layer(x)

x = base_model(x, training=False)               # Set it to inference mode
x = keras.layers.GlobalAveragePooling2D()(x)    # Pooling
x = keras.layers.Dropout(0.4)(x)                # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.9),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 2
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [ ]:
### THIS WILL TAKE A WHILE ON CPUs ###
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 2
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
### THIS WILL TAKE A WHILE ON CPUs ###
